<a href="https://colab.research.google.com/github/anuragbantu/DL-Assignment-2/blob/main/Assignment2_partB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split


In [ ]:
from torchvision.models import efficientnet_v2_s

model = efficientnet_v2_s(pretrained=True)



In [ ]:
#if running on google colab, use this cell

import os
import zipfile

zip_path = '/content/drive/MyDrive/nature_12K.zip'

extract_path = '/content/iNaturalist'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)

Dataset extracted to: /content/iNaturalist


In [ ]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),   # match expected input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean
                         [0.229, 0.224, 0.225])  # ImageNet std
])


In [ ]:
#load the data
train_dir = '/content/iNaturalist/inaturalist_12K/train'
test_dir = '/content/iNaturalist/inaturalist_12K/val'

train_dataset = ImageFolder(root=train_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)


In [ ]:
# Modify final layer to match number of classes
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)


In [ ]:
# Load the full training dataset
full_train_dataset = ImageFolder(root=train_dir, transform=transform)

# Calculate split sizes
val_size = int(0.2 * len(full_train_dataset))
train_size = len(full_train_dataset) - val_size

# Split the dataset
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Load the test set
test_dataset = ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:
for name, child in model.features.named_children():
    if int(name) < 4:  # Freeze blocks 0 to 3
        for param in child.parameters():
            param.requires_grad = False
    else:
        for param in child.parameters():
            param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True


In [ ]:
import torch

#criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
#for freezing use this
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


In [ ]:
import copy

# Save the initial state of the frozen layers (just one block to check)
initial_weights = copy.deepcopy(model.features[0][0].weight.data.clone())


In [ ]:
#Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (preds == labels).sum().item()

    train_acc = 100 * correct_train / total_train
    train_loss = running_loss / len(train_loader)

    # --- Validation ---
    model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_outputs = model(val_inputs)
            _, val_preds = torch.max(val_outputs, 1)
            total_val += val_labels.size(0)
            correct_val += (val_preds == val_labels).sum().item()

    val_acc = 100 * correct_val / total_val

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Training Loss: {train_loss:.4f} | Training Accuracy: {train_acc:.2f}%")
    print(f"  Validation Accuracy: {val_acc:.2f}%\n")

Epoch 1/5
  Training Loss: 2.0342 | Training Accuracy: 39.51%
  Validation Accuracy: 61.23%

Epoch 2/5
  Training Loss: 1.6230 | Training Accuracy: 61.64%
  Validation Accuracy: 66.28%

Epoch 3/5
  Training Loss: 1.3926 | Training Accuracy: 65.08%
  Validation Accuracy: 67.28%

Epoch 4/5
  Training Loss: 1.2613 | Training Accuracy: 65.44%
  Validation Accuracy: 69.03%

Epoch 5/5
  Training Loss: 1.1766 | Training Accuracy: 66.89%
  Validation Accuracy: 69.78%



In [ ]:
updated_weights = model.features[0][0].weight.data
# Compare
if torch.equal(initial_weights, updated_weights):
    print("Frozen layer weights have not changed.")
else:
    print("Frozen layer weights have changed")
# Compare

Frozen layer weights have not changed.


In [ ]:
#replace validation set with test in the training loop?

In [ ]:
# --- Evaluate on test set ---
model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
        test_outputs = model(test_inputs)
        _, test_preds = torch.max(test_outputs, 1)
        total_test += test_labels.size(0)
        correct_test += (test_preds == test_labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 70.10%


In [ ]:
#for epochs = 5
#without freezing test acc = 84.85
#with freezing all layers except last = 70.1
#only pre trained model without finetuning on naturalist = 13.00